# 2-2 extract api list from java code

- step1 java code 코드 수집
- step2 api에서 정규식을 적용하여 전체 리스트 및 갯수 확인
- step3 api에서 비번하게 나타나는 10000개의 리스트를 가지고 malware, benign의 비율 확인
- step4 10000개의 리스트 중에서 영향도 있는 리스트만 각출
- step5 각 apk 파일들의 api 수집 및 정리


## step1 java code 코드 수집

In [1]:
import os

dir_names = ["Benign_samples_1st_1500", "Malware_samples_1st_500"]
dir_list = dict()

i = 0
print( ".",end =' ')
for dir_name in dir_names:
    dir_name = dir_name + "_apktools"
    if dir_name.startswith('Benign'):
        tag = 'Benign'
    else :
        tag = 'Malware'
        
    for root, dirs, files in os.walk(dir_name):
        for f in files:
            if f.endswith(".java"):
                dir_list[root + '/' + f ] = tag
                i+=1
                if i%10000 == 0:
                    print( 'Count: {}                 \r'.format(i),end ='')
            
            
                    
print( 'Count: {}                 \r'.format(i),end ='')

In [2]:
#dir_list.keys()[:10]
a = dir_list.keys()
a = list(a)
a[:10]

['Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/out/com/appsbar/CBFerreterias140547/LauncherActivity.java',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/out/com/appsbar/CBFerreterias140547/AppIDActivity.java',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/out/com/appsbar/CBFerreterias140547/R.java',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/out/com/appsbar/CBFerreterias140547/ActivityWMenu.java',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/out/com/appsbar/CBFerreterias140547/Adapters/MediaListViewAdapter.java',
 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/out/com/appsbar/CBFerreterias140547/Adapters/EventsAdapter.java',
 'Benign_samples_1st_1500_apktools/b54a9bd

In [7]:
import re
import nltk
from multiprocessing import Pool
from collections import Counter
import numpy as np

# Create three Counter objects to store positive, negative and total counts
global benign_list
global malware_list
global total_counts

benign_list = Counter()
malware_list = Counter()
total_counts = Counter()


def file2strings(key):
    with open(key, encoding = "ISO-8859-1") as f:
        data = f.read()
        #data = str(data)#.encode("ascii","ignore")
        #data = data.decode("ascii","ignore")
        data = data.lower()
        
    pat = re.compile("([a-zA-Z0-9_]+)\(.*\)")
    return dir_list[key], pat.findall(data)

with Pool(32) as p:
    count = len(dir_list)
    print ("start")
    i = 0
    for key, data in p.imap(file2strings, list(dir_list.keys())):
        
            if  key == 'Benign':
                for word in data:
                    benign_list[word] += 1
                    total_counts[word] += 1
                    
            else :
                for word in data:
                    malware_list[word] += 1
                    total_counts[word] += 1
                    
            i += 1    
            if i%100 == 0:
                print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')

    print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')
    

start


In [8]:
len(total_counts)

320832

In [36]:
topCut = total_counts.most_common()[:100000]
topCut[-1]

('setsubjectkeyidentifier', 6)

## step3 smali 코드에서빈번하게 나타나는 10000개의 리스트를 가지고 malware, benign의 비율 확인

In [37]:
benign_list.most_common()[:100]

[('makeinsn', 37913376),
 ('makeinsnbody', 37872430),
 ('addarg', 37871883),
 ('makeinvoke', 18937137),
 ('inlinemethod', 18935537),
 ('a', 577305),
 ('append', 280505),
 ('put', 225387),
 ('equals', 195717),
 ('get', 182281),
 ('add', 168088),
 ('b', 163289),
 ('findviewbyid', 119140),
 ('d', 116362),
 ('super', 116277),
 ('tostring', 116175),
 ('valueof', 105030),
 ('e', 103905),
 ('size', 100495),
 ('getinstance', 92601),
 ('c', 91148),
 ('close', 90927),
 ('action', 87371),
 ('length', 82569),
 ('print', 73420),
 ('illegalargumentexception', 70035),
 ('setvisibility', 59908),
 ('arraylist', 59214),
 ('write', 58047),
 ('settext', 56633),
 ('getstring', 52179),
 ('run', 50396),
 ('println', 44696),
 ('i', 43533),
 ('onclick', 42981),
 ('this', 41967),
 ('putextra', 41563),
 ('remove', 41258),
 ('v', 41212),
 ('stringbuilder', 40118),
 ('isempty', 39611),
 ('illegalstateexception', 38552),
 ('oncreate', 37835),
 ('addview', 35945),
 ('set', 35254),
 ('clear', 34520),
 ('hasnext', 343

In [38]:
malware_list.most_common()[:100]

[('makeinsn', 3277848),
 ('makeinsnbody', 3273086),
 ('addarg', 3273020),
 ('inlinemethod', 1636516),
 ('makeinvoke', 1636514),
 ('a', 527336),
 ('b', 147747),
 ('put', 100732),
 ('append', 89459),
 ('equals', 84413),
 ('c', 83031),
 ('close', 82200),
 ('d', 78846),
 ('printstacktrace', 65271),
 ('e', 64063),
 ('add', 63272),
 ('get', 58589),
 ('valueof', 55477),
 ('tostring', 44738),
 ('i', 43951),
 ('size', 38726),
 ('getstring', 38418),
 ('findviewbyid', 35258),
 ('putextra', 34599),
 ('setvisibility', 34112),
 ('super', 32994),
 ('addview', 32152),
 ('length', 30811),
 ('write', 27763),
 ('f', 24860),
 ('arraylist', 24660),
 ('run', 24632),
 ('getsharedpreferences', 23752),
 ('settext', 23276),
 ('stringbuilder', 22441),
 ('print', 22102),
 ('intent', 21856),
 ('getsystemservice', 21244),
 ('g', 20423),
 ('onclick', 19982),
 ('getint', 18421),
 ('layoutparams', 18262),
 ('putstring', 17843),
 ('h', 17451),
 ('substring', 16002),
 ('setonclicklistener', 15162),
 ('startactivity', 15

In [39]:
benign_malware_ratios = Counter()

# Calculate the ratios of positive and negative uses of the most common words
# Consider words to be "common" if they've been used at least 100 times
for term, cnt in list(topCut):
    benign_malware_ratio = benign_list[term]+3 / (float(malware_list[term]+1)*3)
    benign_malware_ratios[term] = benign_malware_ratio

for word,ratio in list(benign_malware_ratios.most_common()):
    benign_malware_ratios[word] = np.log(ratio)

word1 = 'addview'
word2 = 'setvisibility'
word3 = 'makeinsn'
print("benign_to_malware ratio for '{}' = {}".format(word1, benign_malware_ratios[word1]))
print("benign_to_malware ratio for '{}' = {}".format(word2, benign_malware_ratios[word2]))
print("benign_to_malware ratio for '{}' = {}".format(word3, benign_malware_ratios[word3]))

benign_to_malware ratio for 'addview' = 10.48974527228322
benign_to_malware ratio for 'setvisibility' = 11.000565331601608
benign_to_malware ratio for 'makeinsn' = 17.450814536547853


In [40]:
# words most frequently seen in a review with a "POSITIVE" label
list(benign_malware_ratios.most_common())[:100]

[('makeinsn', 17.450814536547853),
 ('makeinsnbody', 17.449733964706098),
 ('addarg', 17.449719521377528),
 ('makeinvoke', 16.756635472733347),
 ('inlinemethod', 16.756550979095483),
 ('a', 13.266126002022945),
 ('append', 12.544346829137812),
 ('put', 12.325574203724567),
 ('equals', 12.184425017326493),
 ('get', 12.113304731521799),
 ('add', 12.03224293086291),
 ('b', 12.003276916045319),
 ('findviewbyid', 11.688054551420731),
 ('d', 11.664461300607545),
 ('super', 11.66373055480704),
 ('tostring', 11.662852954145578),
 ('valueof', 11.56200130278844),
 ('e', 11.551232299275043),
 ('size', 11.517863254256799),
 ('getinstance', 11.436055220560542),
 ('c', 11.420239838144129),
 ('close', 11.417812265897744),
 ('action', 11.377918754634363),
 ('length', 11.321389586795025),
 ('print', 11.203951657667043),
 ('illegalargumentexception', 11.156750397255051),
 ('setvisibility', 11.000565331601608),
 ('arraylist', 10.988913280084864),
 ('write', 10.969008306824533),
 ('settext', 10.9443471339

In [41]:
# words most frequently seen in a review with a "NEGATIVE" label
list(reversed(benign_malware_ratios.most_common()))[:100]

[('sdkutils', -8.0580108008020854),
 ('restartsdk', -7.6221745948176221),
 ('lauchnewhttptask', -7.1929342212157996),
 ('getdatasharedprefrences', -7.1846291527173145),
 ('____', -7.1724245771248452),
 ('ymbubqk', -6.9440872082295275),
 ('airpush', -6.9147308927185627),
 ('nativetileat', -6.7934661325800096),
 ('idfhn', -6.6758232216348476),
 ('appconnect', -6.6306833856423717),
 ('putsettingsvalue', -6.4738906963522744),
 ('hpceruvju', -6.4614681763537174),
 ('setrecvadstatus', -6.4536249988926917),
 ('counterreceiver', -6.4473058625412127),
 ('getinstancenoconnect', -6.4425401664681985),
 ('httppostdatatask', -6.4361503683694279),
 ('isshowoptindialog', -6.3543700407973507),
 ('setapplicationdetails', -6.3508857167147399),
 ('delivernotification', -6.3189681137464344),
 ('netutil', -6.2422232654551655),
 ('mkfkejkpu', -6.2265366692874657),
 ('respclicklist', -6.1882641230825897),
 ('lqsxoock', -6.1820849067166312),
 ('resetrollover_pri', -6.1441856341256456),
 ('shortcuticonresource'

In [44]:
rank = 5000
print((benign_malware_ratios.most_common())[rank])
print(list(reversed(benign_malware_ratios.most_common()))[rank])

('endofstream', 5.5612309306722949)
('advertisementaction', -2.0794415416798357)


### 영향도 있는 benign_malware_ratios 리스트 저장
benign과 malware에 있는 영향도 있는 benign_malware_ratios를 저장한다.

In [45]:
import pickle

with open('benign_malware_fuction_ratios.pkl', 'wb') as f:
    pickle.dump(benign_malware_ratios, f)

In [46]:
import pickle

with open('benign_malware_fuction_ratios.pkl', 'rb') as f:
    benign_malware_ratios = pickle.load(f)

from collections import Counter 
benign_malware_ratios.most_common()

[('makeinsn', 17.450814536547853),
 ('makeinsnbody', 17.449733964706098),
 ('addarg', 17.449719521377528),
 ('makeinvoke', 16.756635472733347),
 ('inlinemethod', 16.756550979095483),
 ('a', 13.266126002022945),
 ('append', 12.544346829137812),
 ('put', 12.325574203724567),
 ('equals', 12.184425017326493),
 ('get', 12.113304731521799),
 ('add', 12.03224293086291),
 ('b', 12.003276916045319),
 ('findviewbyid', 11.688054551420731),
 ('d', 11.664461300607545),
 ('super', 11.66373055480704),
 ('tostring', 11.662852954145578),
 ('valueof', 11.56200130278844),
 ('e', 11.551232299275043),
 ('size', 11.517863254256799),
 ('getinstance', 11.436055220560542),
 ('c', 11.420239838144129),
 ('close', 11.417812265897744),
 ('action', 11.377918754634363),
 ('length', 11.321389586795025),
 ('print', 11.203951657667043),
 ('illegalargumentexception', 11.156750397255051),
 ('setvisibility', 11.000565331601608),
 ('arraylist', 10.988913280084864),
 ('write', 10.969008306824533),
 ('settext', 10.9443471339

In [47]:
import pandas as pd
limit = 5000
df = pd.DataFrame(list(benign_malware_ratios.most_common())[:limit] + list(benign_malware_ratios.most_common())[-limit:])
df

,0,1
0,makeinsn,17.450815
1,makeinsnbody,17.449734
2,addarg,17.449720
3,makeinvoke,16.756635
4,inlinemethod,16.756551
5,a,13.266126
6,append,12.544347
7,put,12.325574
8,equals,12.184425
9,get,12.113305


In [48]:
word_list = list(df[0])
len(word_list)
'makeinsn' in word_list

True

### Benign 일반 파일 csv 파일 불러오기

## step5 각 apk 파일들의 4gram 수집 및 정리
apk_word_list
파일로부터 데이터를 불러와서 해시 값에 따라 counter 값 추가

In [50]:
word_function_list = dict()

def file2strings(key):
    
    word_count = Counter()
    filename = key.split("/")[1] # 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/ActivityWMenu$FbLoginDialogListener.smali'
    
    with open(key, encoding = "ISO-8859-1") as f:
        data = f.read()
        #data = str(data)#.encode("ascii","ignore")
        #data = data.decode("ascii","ignore")
        data = data.lower()
        
    pat = re.compile("([a-zA-Z0-9_]+)\(.*\)")
    
    for word in pat.findall(data):
        if word in word_list:
            word_count[word] += 1
            
    return filename, dict(word_count.most_common())

with Pool(32) as p:
    print ("start")
    i = 0
    count = len(dir_list)
    
    for filename, word_count in p.imap(file2strings, list(dir_list.keys())):
        word_function_list[filename] = word_count
        
        i += 1    
        if i %100 == 0:
            print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')
     
    print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')

start


In [51]:
df1 = pd.DataFrame(word_function_list).T
df1

,a,a_,ab,abortanimation,abs,accept,accessibilityserviceinfoicsimpl,accessibilityserviceinfostubimpl,acquire,activate,...,writetoparcel,writeutf,writevalues,wrongtype,xor,y,ymbubqk,yytoken,z,zipexception
000b14d01e2416f3c4b8d260a20021ec477f543993e1f4041f0efd7c95f00bfb,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002f4f25130c0090b7b8ece3e3b41a3e082e224f77159e665d67e3d64a358c85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
004679da462ba3ca1fff77c8d8e1f5a057f2eaf30f0feb3f07320992619ff9d1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0081079f409a40cbb81c5a3b47dd56242bcda516aeb79ec13a13b0e913162d39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00b04c9a5966b0d64c135ee3894a9410ecafb097a6b342619274b16d7fea4c4e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00b28b5428689c82cda142420f220c2be821eaf913fc951187a7099f39e6fa0e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00b838179a351a6cfd77af944c35a1758dfaa0a6ddba04fc423f3d6409e50ee4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00cb6d24d37ee40bef1c9ac231880b50bcf45c554ef863ed4f16554de6720885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00ea77eb6f076fe87adb3cd9d39bcb3e8e487529faed7c8853c5a821be801e0a,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
012432a7b76a23d68d18fbcd88a2cf13c0c89e94351e55f65582c18738197a82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df1.describe()

,a,a_,ab,abortanimation,abs,accept,accessibilityserviceinfoicsimpl,accessibilityserviceinfostubimpl,acquire,activate,...,writetoparcel,writeutf,writevalues,wrongtype,xor,y,ymbubqk,yytoken,z,zipexception
count,328.000000,20.000000,4.0,1.0,23.000000,6.0,7.0,7.0,6.000000,9.000000,...,13.000000,2.0,20.000000,20.000000,2.0,77.000000,2.0,5.0,2.0,3.0
mean,5.597561,4.800000,1.0,2.0,2.086957,1.0,2.0,2.0,1.166667,2.555556,...,1.769231,1.0,58.700000,21.450000,4.0,1.103896,66.0,1.0,1.0,2.0
std,6.649568,0.894427,0.0,NaN,1.806882,0.0,0.0,0.0,0.408248,0.881917,...,0.832050,0.0,60.500631,0.686333,0.0,0.307127,0.0,0.0,0.0,0.0
min,1.000000,1.000000,1.0,2.0,1.000000,1.0,2.0,2.0,1.000000,1.000000,...,1.000000,1.0,3.000000,20.000000,4.0,1.000000,66.0,1.0,1.0,2.0
25%,2.000000,5.000000,1.0,2.0,1.000000,1.0,2.0,2.0,1.000000,3.000000,...,1.000000,1.0,22.500000,21.000000,4.0,1.000000,66.0,1.0,1.0,2.0
50%,3.000000,5.000000,1.0,2.0,1.000000,1.0,2.0,2.0,1.000000,3.000000,...,2.000000,1.0,34.500000,21.000000,4.0,1.000000,66.0,1.0,1.0,2.0
75%,8.000000,5.000000,1.0,2.0,2.000000,1.0,2.0,2.0,1.000000,3.000000,...,2.000000,1.0,82.000000,22.000000,4.0,1.000000,66.0,1.0,1.0,2.0
max,61.000000,5.000000,1.0,2.0,8.000000,1.0,2.0,2.0,2.000000,3.000000,...,4.000000,1.0,253.000000,23.000000,4.0,2.000000,66.0,1.0,1.0,2.0


In [54]:
df1.to_csv('1st_java_function.csv')